# Заняття 6

## Мета:
Навчити (finetune) попередньо треновану велику мовну модель (LLM) на конкретному наборі даних для досягнення високої точності у вирішенні конкретної задачі, а також провести оцінку її ефективності.

---

## Кроки для виконання завдання:

### 1. Підготовка середовища:
- Встановіть необхідні бібліотеки: Hugging Face Transformers, TensorFlow або PyTorch, numpy, pandas.
- Визначте та підготуйте доступ до обчислювальних ресурсів (локальний комп’ютер або сервер із GPU).

### 2. Вибір попередньо навченої LLM та набору даних:
- Оберіть відповідну LLM, таку як Llama, Gemma, GPT-2, BERT, T5, або іншу модель із бібліотеки Hugging Face.
- Завантажте або підготуйте набір даних.
- Переконайтесь, що дані добре структуровані та підготовлені для навчання (наприклад, файли CSV або JSON).

### 3. Попередня обробка даних:
- Виконайте попередню обробку тексту, включаючи токенізацію, видалення зайвих символів або перетворення в потрібний формат.
- Використайте вбудовані токенізатори з бібліотеки Transformers для обраної моделі.

### 4. Налаштування (finetuning) моделі:
- Завантажте обрану попередньо навчану модель.
- Визначте гіперпараметри для навчання: розмір батчу, кількість епох, коефіцієнт навчання.
- Виконайте налаштування моделі на вашому наборі даних за допомогою API бібліотеки Transformers (Hugging Face).

### 5. Оцінка ефективності моделі:
- Розділіть ваш набір даних на навчальну та тестову (або валідаційну) вибірки.
- Після навчання обчисліть показники ефективності, BLEU та ROUGE для задач генерації тексту.
- Проаналізуйте результати та порівняйте їх із базовою (до навчання) продуктивністю моделі.

### 6. Оптимізація та повторне навчання:
- На основі отриманих результатів, за потреби, налаштуйте гіперпараметри або передобробку даних.
- Повторіть навчання моделі та порівняйте результати після оптимізації.

### 7. Документування результатів:
- Зробіть висновки щодо ефективності моделі після налаштування.
- Опишіть, які покращення були досягнуті та в яких випадках модель показала найкращі результати.
- За можливості, надайте візуалізацію метрик для кращого розуміння результатів.

---

## Мінімальні вимоги:
- Налаштувати (finetune) попередньо навчений LLM на конкретному наборі даних.
- Оцінити ефективність моделі за допомогою відповідних метрик.

---

## Формат виконання:
- `.ipynb` блокнот код в `.py` з інференсом для запуску моделі та короткою документацією.


## GPT 4o mini Fine Tunning

### Підготовка даних

#### Конвертація віршів у json

Маємо текстовий файл зі структурою:

```
-----
---
<назва віршу>
---
<вірш 
на 
багато
рядків>
-----
---
<назва віршу 2>
---
<вірш 2
на 
багато
рядків>
...
```

і конвертуємо його у формат JSON:

```
[
  {"name": "Назва віршу", "content": "Вірш\nрядок1\nрядок2 ..."},
  {"name": "Назва віршу 2", "content": "Вірш2\nрядок1\nрядок2 ..."},
  ...
]
```

Після виконання цього коду у файлі `poems.json` буде створено масив віршів у бажаному форматі.

In [3]:
!cd poems && python split.py

#### Конвертація у формат для навчання jsonl

Маємо JSON-файл у форматі:

```json
[
  {"name": "Назва віршу", "content": "Вірш\nрядок1\nрядок2 ..."},
  {"name": "Назва віршу 2", "content": "Вірш2\nрядок1\nрядок2 ..."},
  ...
]
```

і конвертуємо його у формат JSONL для навчання моделі. Кожен рядок JSONL-файлу міститиме об'єкт з полем `messages`, де:

- `"role": "system"`: фіксоване повідомлення
- `"role": "user"`: запит з вказівкою назви вірша (якщо у назві немає рядка `+ + +`)
- `"role": "assistant"`: оригінальний текст вірша


##### Пояснення

- Код відкриває вхідний JSON-файл зі списком віршів.
- Для кожного вірша:
  - Формується масив `messages`, що містить три повідомлення: 
    - `system` (статичне), 
    - `user` (з вбудованою назвою вірша, якщо немає `+++`),
    - `assistant` (текст вірша).
- Результат записується у файл `poems.jsonl`, де кожен рядок — окремий JSON-об’єкт.

Таким чином, модель зможе використовувати цей файл для подальшого донавчання у потрібному форматі.

In [4]:
!cd poems/ && python convert.py

In [6]:
filename = "poems/poems.jsonl"

with open(filename, "r", encoding="utf-8") as f:
    for i in range(10):
        line = next(f, None)
        if line is None:
            break
        print(line, end="")

{"messages": [{"role": "system", "content": "SerGPT Zhadan - найкращий поет української сучасності"}, {"role": "user", "content": "Напиши красивий вірш'}"}, {"role": "assistant", "content": "Ще нічого немає. Зелена ніч,\n\nі в кожної тиші міра своя.\n\nІ, знаючи, скільки потрібно сторіч,\n\nщоби з’явилась найперша річ,\n\nвін вимовляє її ім’я.\n\n\n\nНемов відчиняє нічне вікно\n\nі ловить напружено кожен рух,\n\nсподіваючись на щось все одно,\n\nі небуття важке полотно\n\nпокірно дається йому до рук.\n\n\n\nІ все, що буде з ними тепер, —\n\nгольфстріми, айсберги мертвих морів,\n\nщоденний рух повітряних сфер,\n\nспів кашалотів і крик химер,\n\nпоява запахів і кольорів,\n\n\n\nкоріння трав і листя дерев,\n\nозерні льоди і пташиний свист,\n\nруди і вугілля натруджений трем,\n\nслухняних тварин шепіт і рев,\n\nжага лунких торговельних міст,\n\n\n\nвогонь, що спалює кораблі,\n\nсмерть на темних шовках знамен,\n\nзгаслі зірки на високому тлі,\n\nтихі мерці в літній землі,\n\nкров, ніби лава

### Опис запуску процесу навчання моделі

1. **Перейдіть у веб-інтерфейс для навчання Fine-tuning**  
   Відкрийте платформу OpenAI за посиланням: [https://platform.openai.com/finetune](https://platform.openai.com/finetune).

2. **Створення нової задачі навчання**  
   Натисніть кнопку **"Create"** у розділі Fine-tuning.

3. **Налаштування параметрів навчання**  
   Заповніть необхідні поля наступним чином:  
   - **Base Model**: Оберіть модель `gpt-4o-mini` зі списку доступних.  
   - **Training Data**: Оберіть **[Upload new]** і завантажте підготовлений файл `poems.jsonl`. Упевніться, що файл відповідає форматам JSONL, вказаним у документації OpenAI.  
   - **Validation Data**: Оберіть **None**, якщо валідаційні дані не потрібні, або завантажте відповідний файл, якщо доступний.  
   - **Hyperparameters (за бажанням)**: Налаштуйте параметри, такі як кількість епох, розмір пакета (batch size), та інші, якщо це необхідно для ваших цілей.

4. **Запуск навчання**  
   Після налаштування всіх параметрів натисніть кнопку **"Create"**, щоб почати процес навчання.

5. **Моніторинг процесу навчання**  
   Дочекайтеся завершення процесу навчання. У цей час можна слідкувати за прогресом у розділі Fine-tuning Tasks на платформі. Ви побачите статуси, такі як **In Progress** або **Completed**, залежно від етапу виконання.

6. **Отримання ідентифікатора моделі**  
   Після завершення навчання знайдіть у списку вашу задачу і скопіюйте **Model ID**. Цей ідентифікатор використовується для інтеграції моделі через API.

7. **Перевірка роботи моделі** (рекомендовано)  
   За допомогою Model ID протестуйте модель через OpenAI Playground або API-запити, щоб впевнитися, що модель відповідає вашим очікуванням.



### Тестування та порівняння моделі з результатами


In [28]:
%pip install openai
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ['SCG_OPENAI_API_KEY'])

system_poem_text = "SerGPT Zhadan - найкращий поет української сучасності"
user_poem_text = "Напиши красивий вірш"

system_analysis_text = "Ти прекрасно розбираєшся в українській літературі, знаєш стилі усіх авторів і можеш визначати поетів по їх особливостям."
user_analysis_text = "Ось тобі вірш, визнач його автора:\n\n"


Note: you may need to restart the kernel to use updated packages.


In [29]:
def message(role, content):
    return {"role": role, "content": [{"type": "text", "text": content}]}


def get_response(model_id, messages):
    response = client.chat.completions.create(
        model=model_id,
        messages=messages,
        response_format={"type": "text"},
        temperature=0.5,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )

    return response.choices[0].message.content

def get_messages(system_text, user_text):
  return [
      message("system", system_text),
      message("user", user_text),
  ]


#### GPT 4o mini



In [30]:
messages = get_messages(system_poem_text, user_poem_text)
base_poem = get_response("gpt-4o-mini", messages)

print(f"GPT 4o mini:\n{base_poem}")

print('---------------ANALYSIS------------------')

messages = get_messages(system_analysis_text, user_analysis_text + base_poem)
base_analysis = get_response("gpt-4o-mini", messages)

print(f"ANALYSIS RESULT\n{base_analysis}")

GPT 4o mini:
У вечірній тіні, де зорі палають,  
Вітри шепочуть казки невідомі,  
Там серце моє, мов птиця, літає,  
В обіймах природи, в обіймах спокою.

Сонце, що сідає, малює на небі  
Рожеві відтінки, мов мрії вітрів,  
І кожен момент, що в серці, як пісня,  
Лунає у тиші, де час не спішить.

Я слухаю шепіт, що ллється з потоків,  
Де квіти, немов таємниці, цвітуть,  
У кожній краплині, у кожному звуці  
Зберігаються спогади, що не забудуть.

О, як же прекрасна ця мить, ця година,  
Коли світ навколо, мов казка, живе,  
І в серці моєму, в тиші безкрайній,  
Розквітає любов, немов ранкова роса.
---------------ANALYSIS------------------
ANALYSIS RESULT
Цей вірш має характерні риси, які можуть вказувати на творчість сучасних українських поетів, зокрема на поезію, що відзначається ліричністю та описами природи. Однак, його стиль і тон можуть також нагадувати про класичних українських поетів, таких як Леся Українка або Павло Тичина, які часто використовували образи природи для вираження

#### GPT 4o mini [Fine-tuned]

In [31]:
messages = get_messages(system_poem_text, user_poem_text)
ft_poem = get_response("ft:gpt-4o-mini-2024-07-18:scg::Ad1s5uFT", messages)

print(f"GPT 4o mini [Fine-tuned]:\n{ft_poem}")

print("---------------ANALYSIS------------------")

messages = get_messages(system_analysis_text, user_analysis_text + ft_poem)
ft_analysis = get_response("gpt-4o-mini", messages)

print(f"ANALYSIS RESULT\n{ft_analysis}")

GPT 4o mini [Fine-tuned]:
Тому що ця зима була такою довгою і холодною,

не було жодного шансу побачити її в цю пору року,

не було жодного шансу на те, що вона сама

знайде час і можливість повернутись,

не було жодного шансу, що вона зможе

забратися зі своїх теплих країв.



І от тепер, коли вона нарешті прийшла,

коли вона нарешті з’явилась,

коли вона нарешті доторкнулась до всіх нас,

коли вона нарешті торкнулась і мене,

я просто не знаю, як мені з цим жити,

я просто не знаю, як мені тепер без неї

пересиджувати цю спеку.



Вона прийшла і зупинилась,

як зупиняється хвиля на мить перед відкатом,

як зупиняється повітря перед грозою,

і я не міг повірити, що вона стоїть так близько від мене,

що я можу дотягнутись до неї,

що вона дихає моїм диханням,

що вона бачить мене, що вона дивиться на мене,

що я їй теж потрібен.



Я просто не знаю, що мені робити тепер,

коли вона піде,

я просто не знаю, як мені без неї жити,

я просто не знаю, як мені без неї

пересиджувати цю спеку